In [2]:
:dep darn
:dep ndarray
:dep ndarray-csv
:dep polars = { version = "x", features = ["csv","describe","fmt","lazy","ndarray","dtype-categorical","to_dummies","regex","strings"]}
:dep smartcore = { version = "0.3.1", features = ["ndarray-bindings"] }

:sccache 1
    
extern crate csv;

use std::io::prelude::*;
use std::fs::*;
use std::path::Path
use ndarray::prelude::*;
use ndarray_csv::Array2Reader;
use ndarray::*;

use polars::prelude::*;
use std::fs::File;

use smartcore::*;
use smartcore::model_selection::train_test_split;
use smartcore::metrics::roc_auc_score;
use smartcore::metrics::precision::Precision;
use smartcore::metrics::accuracy::Accuracy;

use smartcore::metrics::Metrics;

use smartcore::metrics::mean_squared_error;
use smartcore::linalg::basic::matrix::DenseMatrix;
use smartcore::linear::logistic_regression::*;
use smartcore::api::{Transformer, UnsupervisedEstimator};
use smartcore::linalg::basic::matrix::DenseMatrix;
use smartcore::preprocessing::numerical;
    



sccache: true. Warning: dynamic linking disabled, use :cache instead to preserve dynamic linking


In [72]:
// Criando curvatura sigmoide em um plano cartesiano
// baseado na regressao linear
// Sigmoid
// p = 1/1+pow(e,-y)
// e = euler = 2.71...

// equacao da reta
// y = b0 + b1 * x
// encontrar o minimo global, do declive da curva


let df = CsvReader::from_path("./db/risco_credit.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

let df= df.clone().lazy();


let original = df.clone().filter(col("risco").str().contains(lit("baixo|alto"), false)).select([
    
        col("*")
        
        ])

    .drop_nulls(None).collect();
println!("DataFrame {:?}",original);


let x = df.clone().filter(col("risco").str().contains(lit("baixo|alto"), false)).select([
    
        col("historia").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("divida").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("garantias").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("renda").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),    
        ])

    .drop_nulls(None).collect();
println!("DataFrame {:?}",x);



let y = df.clone().filter(col("risco").str().contains(lit("baixo|alto"), false)).select([col("risco")])
    .with_columns([col("risco").alias("risco_cat").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical()])
    .drop_nulls(None)
    .collect();
println!("DataFrame y {:?}",y);

let y = y?.column("risco_cat")?.clone();
{
let ndarray_x = x?.clone().to_ndarray::<Float32Type>(IndexOrder::C).unwrap();

let ndarray_y:Vec<u32> = y.u32()?.into_iter().filter_map(|opt_str| opt_str).collect();

    
let (x_train, x_test, y_train, y_test) = train_test_split(&ndarray_x, &ndarray_y, 0.3, true, None);
println!("x_train {:?}",x_train);
println!("x_test {:?}",x_test);
println!("y_train {:?}",y_train);
println!("y_test {:?}",y_test);


let test_values = array![[0.0,0.0,0.0,0.0],[1.0,1.0,0.0,2.0]];


    
    
let lr = LogisticRegression::fit(&x_train, &y_train, Default::default()).and_then(|lr| lr.predict(&x_test)).unwrap();
    println!("y_test {:?}",lr);



fn u32_to_f32_vec_ary(v: Vec<u32>) -> Vec<f32> {
    v.iter().map(|&x| x as f32).collect()
}


let y_test_f = u32_to_f32_vec_ary(y_test.clone());
let y_hat_f = u32_to_f32_vec_ary(lr.clone());
 let accuracy: f64 = Accuracy::new().get_score(&y_test_f, &y_hat_f);
println!("Accuracy: {:?}", accuracy);    

}

DataFrame Ok(shape: (11, 5)
┌──────────────┬────────┬───────────┬──────────┬───────┐
│ historia     ┆ divida ┆ garantias ┆ renda    ┆ risco │
│ ---          ┆ ---    ┆ ---       ┆ ---      ┆ ---   │
│ str          ┆ str    ┆ str       ┆ str      ┆ str   │
╞══════════════╪════════╪═══════════╪══════════╪═══════╡
│ ruim         ┆ alta   ┆ nenhuma   ┆ 0_15     ┆ alto  │
│ desconhecida ┆ alta   ┆ nenhuma   ┆ 15_35    ┆ alto  │
│ desconhecida ┆ baixa  ┆ nenhuma   ┆ acima_35 ┆ alto  │
│ desconhecida ┆ baixa  ┆ nenhuma   ┆ acima_35 ┆ baixo │
│ desconhecida ┆ baixa  ┆ adequada  ┆ acima_35 ┆ baixo │
│ …            ┆ …      ┆ …         ┆ …        ┆ …     │
│ boa          ┆ baixa  ┆ nenhuma   ┆ acima_35 ┆ baixo │
│ boa          ┆ alta   ┆ adequada  ┆ acima_35 ┆ baixo │
│ boa          ┆ alta   ┆ nenhuma   ┆ 0_15     ┆ alto  │
│ boa          ┆ alta   ┆ nenhuma   ┆ acima_35 ┆ baixo │
│ ruim         ┆ alta   ┆ nenhuma   ┆ 15_35    ┆ alto  │
└──────────────┴────────┴───────────┴──────────┴───────┘)
Da

()

In [73]:
let df = CsvReader::from_path("./db/census.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

let df= df.clone().lazy();



let x_census = df.clone().select([col("*").exclude(["income"])])
    .with_columns([
        col("workclass").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("final-weight").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("education").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("marital-status").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("occupation").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("relationship").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("race").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("sex").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("hour-per-week").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("capital-loss").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("capital-gain").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("education-num").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),


        col("native-country").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),        
    ])
    .drop_nulls(None).collect();
        println!("train {:?}",x_census);


let y_census = df.clone()
    .select([col("income").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical()
             // .cast(DataType::Float64)
            ])
    .drop_nulls(None)
    .collect();

let y_census = y_census?.column("income")?.clone();


// let dummies_x = x_census?.clone().to_dummies(None, false).unwrap();

let ndarray_x = x_census?.clone().to_ndarray::<Float32Type>(IndexOrder::C).unwrap();

// let ndarray_y:Vec<String> =   y_census.str()?.
//                                     into_iter().
//                                     filter_map(|opt_str| opt_str.map(|s| s.to_string()))
//                                     .collect();
let ndarray_y:Vec<u32> = y_census.u32()?.into_iter().filter_map(|opt_str| opt_str).collect();
{



    
let (x_train, x_test, y_train, y_test) = train_test_split(&ndarray_x, &ndarray_y, 0.2, false, None);

let lr = LogisticRegression::fit(&x_train, &y_train, Default::default()).unwrap();
let y_hat = lr.predict(&x_test).unwrap();
    println!("predict: {:?}", y_test); 

println!("predict: {:?}", y_hat); 


fn u32_to_f32_vec_ary(v: Vec<u32>) -> Vec<f32> {
    v.iter().map(|&x| x as f32).collect()
}


let y_test_f = u32_to_f32_vec_ary(y_test.clone());
let y_hat_f = u32_to_f32_vec_ary(y_hat.clone());

    
println!("MSE: {}", mean_squared_error(&y_test_f, &y_hat_f));   
let score: f64 = Precision::new().get_score(&y_test_f, &y_hat_f);
println!("Precision: {:?}", score); 

 let accuracy: f64 = Accuracy::new().get_score(&y_test_f, &y_hat_f);
println!("Accuracy: {:?}", accuracy);    
}

train Ok(shape: (32_561, 14)
┌─────┬───────────┬────────────┬───────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ age ┆ workclass ┆ final-weig ┆ education ┆ … ┆ capital-ga ┆ capital-lo ┆ hour-per-w ┆ native-cou │
│ --- ┆ ---       ┆ ht         ┆ ---       ┆   ┆ in         ┆ ss         ┆ eek        ┆ ntry       │
│ i64 ┆ u32       ┆ ---        ┆ u32       ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│     ┆           ┆ u32        ┆           ┆   ┆ u32        ┆ u32        ┆ u32        ┆ u32        │
╞═════╪═══════════╪════════════╪═══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 39  ┆ 0         ┆ 0          ┆ 0         ┆ … ┆ 0          ┆ 0          ┆ 0          ┆ 0          │
│ 50  ┆ 1         ┆ 1          ┆ 0         ┆ … ┆ 1          ┆ 0          ┆ 1          ┆ 0          │
│ 38  ┆ 2         ┆ 2          ┆ 1         ┆ … ┆ 1          ┆ 0          ┆ 0          ┆ 0          │
│ 53  ┆ 2         ┆ 3          ┆ 2         ┆ … ┆ 1          ┆ 

()